## 리그 오브 레전드 리그별 경기 결과 크롤링
  
https://qwer.gg/ko/leagues/{league}/{year}/{season}

+ 리그 : LCK(한국), LPL(중국), LCS(북미), LES(유럽)
+ 연도 : 2015 ~
+ 시즌 : 스프링, 서머

In [1]:
#-*- coding: utf-8 -*-
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains

import time
import os

import warnings
warnings.filterwarnings("ignore")

In [3]:
options = webdriver.ChromeOptions()
#options.headless = True
options.add_argument('window_size=1920x1080')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36')

browser = webdriver.Chrome(options=options)

#qwer 사이트에서 LCK 2015년 서머시즌 페이지 열기
league = 'LCK'
year = '2016'
season = 'Summer'
url = f'https://qwer.gg/ko/leagues/{league}/{year}/{season}'

browser.get(url)

In [ ]:
#시즌 내 경기타입 선택
button = browser.find_element(By.ID, 'headlessui-menu-button-16')
button.click()

season_types = browser.find_element(By.ID, 'headlessui-menu-items-17').find_elements(By.TAG_NAME, 'div')
print(season_types[1].get_attribute('innerText'))

season_types[0].click()

Regular season


In [4]:
#경기결과 클릭
button = browser.find_element(By.ID, 'headlessui-tabs-tab-10')
button.click()   
time.sleep(3)

In [ ]:
#각 매치 결과 페이지 href 크롤링
buttons = browser.find_elements(By.TAG_NAME, 'button')

for i in reversed(buttons[16:]):   #매치 별 detail 버튼 누르기
    i.click()
    
a_tag = browser.find_elements(By.TAG_NAME, 'a')
match_link = [i.get_attribute('href') for i in a_tag][17:-5]

#txt 파일에 저장
filename = f'{league}_{year}_{season}_match_links.txt'
with open(filename, 'w') as file:
    file.write('\n'.join(match_link))

In [ ]:
#위 과정을 함수로 만들기
def get_match_url(league, year, season):
    for type_idx in range(4):
        options = webdriver.ChromeOptions()
        options.headless = True   #브라우저 창 띄우지 않고 백그라운드에서 진행
        options.add_argument('window_size=1920x1080')
        options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36')
        options.add_experimental_option("detach", True)
        browser = webdriver.Chrome(options=options)

        url = f'https://qwer.gg/ko/leagues/{league}/{year}/{season}'
        
        try:
            browser.get(url)
            time.sleep(3)
            
            button = browser.find_element(By.ID, 'headlessui-menu-button-16')  #상세시즌 사이드바
            button.click()
            
            season_types = browser.find_element(By.ID, 'headlessui-menu-items-17').find_elements(By.TAG_NAME, 'div')   #상세시즌 내역
                    
            season_type = season_types[type_idx]     
            season_name = season_type.get_attribute('innerText')  #txt 파일 저장 시 이름에 사용
                        
            season_type.click()
                
            button = browser.find_element(By.ID, 'headlessui-tabs-tab-10')
            button.click()   
            time.sleep(1)
                    
            buttons = browser.find_elements(By.TAG_NAME, 'button')

            for i in reversed(buttons[16:]):   #매치 별 detail 버튼 누르기
                i.click()
            time.sleep(1)
                    
            a_tag = browser.find_elements(By.TAG_NAME, 'a')
            match_link = [i.get_attribute('href') for i in a_tag][17:-5]

            #txt 파일에 저장
            filename = f'{league}_{year}_{season}_{season_name}_match_links.txt'
            with open(filename, 'w') as file:
                file.write('\n'.join(match_link))
            time.sleep(1)
            browser.quit()
        
        except:
            browser.quit()    

In [ ]:
#약 80분 소요
leagues = ['LCK', 'LPL', 'LCS', 'LEC']
years = [str(i) for i in range(2015, 2023)]
seasons = ['Spring', 'Summer']

for league in leagues:
    for year in years:
        for season in seasons:
            get_match_url(league, year, season)

In [ ]:
#txt 파일 내용 불러오는 함수
def get_txt_url(path):
    txt_file_list = [file for file in os.listdir(path) if file.endswith('txt')]
    return(txt_file_list)
    
txt_list = get_txt_url('./')    #현재 디렉토리 내 txt 파일들(116개)
len(txt_list)  

116

In [5]:
import pandas as pd
url = pd.read_csv('LCK_2019_Spring_Promotion_match_links.txt', header=None)

url_list = [i for i in url.iloc[:,0]]
url_list

['https://qwer.gg/ko/matches/16912/mvp-vs-btc-2018-09-22',
 'https://qwer.gg/ko/matches/16913/bbq-vs-mvp-2018-09-21',
 'https://qwer.gg/ko/matches/16914/dk-vs-btc-2018-09-20',
 'https://qwer.gg/ko/matches/16915/bbq-vs-dk-2018-09-18',
 'https://qwer.gg/ko/matches/16916/mvp-vs-btc-2018-09-18']

In [6]:
url_list[0]

'https://qwer.gg/ko/matches/16912/mvp-vs-btc-2018-09-22'

In [57]:
#url 내에서 상세 정보 크롤링 해보기
options = webdriver.ChromeOptions()
#options.headless = True
options.add_argument('window_size=1920x1080')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36')
browser = webdriver.Chrome(options=options)

url = url_list[0]
browser.get(url)

In [45]:
#경기일
date = url[-10:]
date

'2018-09-22'

In [8]:
#팀명
team1_name = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[1]/div[2]/div[1]/strong').get_attribute('innerText')
team2_name = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[1]/div[2]/div[3]/strong').get_attribute('innerText')

team1_name, team2_name

('MVP', 'BtC')

In [9]:
#세트 선택
set_list = browser.find_elements(By.CLASS_NAME, 'shrink-0')  

set_1 = set_list[0]
set_1.click()

In [10]:
#경기별 영상 url
videos = browser.find_element(By.XPATH, '//*[@id="__next"]/div[3]/div/div/div[1]/div[2]/div/div')
video_urls = videos.find_elements(By.TAG_NAME, 'figure')

video_url_list = []
for i in video_urls:
    i.click()
    video_url_list.append(browser.find_element(By.TAG_NAME, 'iframe').get_attribute('src'))
    
video_url_list

['https://www.youtube.com/embed/Ro3fI5U0fO8',
 'https://www.youtube.com/embed/gTI7vo_a8DI',
 'https://www.youtube.com/embed/O7WC58JYISQ',
 'https://www.youtube.com/embed/DJjf0vdpA9c',
 'https://www.youtube.com/embed/0Lc-1SvSPI4',
 'https://www.youtube.com/embed/pNrgoPvpQuQ']

In [11]:
#밴
banned = browser.find_elements(By.TAG_NAME, 'a')

team1_banned = banned[25:30]
team2_banned = banned[35:40]

team1_banned = [banned_champ.find_elements(By.TAG_NAME, 'img') for banned_champ in team1_banned]
team2_banned = [banned_champ.find_elements(By.TAG_NAME, 'img') for banned_champ in team2_banned]

team1_banned = [banned_champ[0].get_attribute('src') for banned_champ in team1_banned]
team2_banned = [banned_champ[0].get_attribute('src') for banned_champ in team2_banned]

team1_banned = [str(banned_champ.split(sep='.')[-2]).split(sep='/')[-1] for banned_champ in team1_banned]
team2_banned = [str(banned_champ.split(sep='.')[-2]).split(sep='/')[-1] for banned_champ in team2_banned]

team1_banned, team2_banned

(['DrMundo', 'Irelia', 'Shen', 'Varus', 'Aatrox'],
 ['Urgot', 'Sion', 'Gragas', 'Gnar', 'Ornn'])

In [12]:
#오브젝트(타워, 억제기, 전령, 드래곤, 장로드래곤, 바론)
team1_object = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[2]/div[1]/div[2]')
team1_object = team1_object.text.split(sep='\n')[1:]
print(team1_object)

team2_object = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[2]/div[2]/div[2]')
team2_object = team2_object.text.split(sep='\n')[1:]
print(team2_object)   

['2', '0', '1', '0', '0', '0']
['10', '2', '0', '5', '1', '1']


In [13]:
#획득 골드
team1_gold = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[3]/div[2]/span[2]').get_attribute('innerText')
team2_gold = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[3]/div[2]/span[3]').get_attribute('innerText')

team1_gold, team2_gold

('62,214', '75,680')

In [66]:
#KDA(kill/death/assist)
team1_kda = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[3]/div[1]/span[2]').get_attribute('innerText')
team2_kda = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[3]/div[3]/span[2]').get_attribute('innerText')

team1_kda, team2_kda

('6 / 12 / 13', '12 / 6 / 31')

In [18]:
#진영(blue/red)
team1_territory = browser.find_element(By.XPATH, '//*[@id="headlessui-tabs-panel-18"]/ul/li[1]/div[1]/div[2]/div/span').get_attribute('innerText')
team2_territory = browser.find_element(By.XPATH, '//*[@id="headlessui-tabs-panel-18"]/ul/li[2]/div[1]/div[2]/div/span').get_attribute('innerText')

team1_territory, team2_territory

('Blue Side', 'Red Side')

In [65]:
#승패 여부(victofy/defeated)
team1_v_d = browser.find_element(By.XPATH, '//*[@id="headlessui-tabs-panel-18"]/ul/li[1]/div[1]/div[1]/strong').get_attribute('innerText')
team2_v_d = browser.find_element(By.XPATH, '//*[@id="headlessui-tabs-panel-18"]/ul/li[2]/div[1]/div[1]/strong').get_attribute('innerText')

team1_v_d, team2_v_d

('Defeat', 'Victory')

In [64]:
#선수별 상세 데이터
data = browser.find_element(By.CLASS_NAME, 'py-4').find_elements(By.TAG_NAME, 'tbody')
player_data = [player_data.find_elements(By.TAG_NAME, 'tr') for player_data in data]   #(2 x 5)
player_data_list = [x for x in player_data[0]] + [x for x in player_data[1]]           #(1 x 10)

final_data = [] 

for i in player_data_list:
    player_data = i.find_elements(By.TAG_NAME, 'td')[0]
    champion = str(player_data.find_element(By.TAG_NAME, 'img').get_attribute('src')).split('.')[-2].split('/')[-1]
    summoner_skill = [i.find_elements(By.TAG_NAME, 'img')[1].get_attribute('title'), #.split(sep='.')[-2].split('/')[-1]
              str(i.find_elements(By.TAG_NAME, 'img')[2].get_attribute('title'))] #.split(sep='.')[-2].split('/')[-1]]
    rune = [str(i.find_elements(By.TAG_NAME, 'img')[3].get_attribute('alt')), #.split(sep='.')[-2].split('/')[-1]
            str(i.find_elements(By.TAG_NAME, 'img')[4].get_attribute('alt'))] #.split(sep='.')[-2].split('/')[-1]
    
    player_data = i.get_attribute('innerText').split(sep='\n\t\n')
    level = player_data[0]
    player_name = player_data[1]
    
    kda = player_data[2].split('\n')[0]
    k, d, a = kda.split(' / ')
    
    damage = player_data[3]
    dpm = player_data[4]
    dtpm = player_data[5]
    sight = player_data[6].split(sep='\n')[0]
    ward_install = player_data[6].split(sep='\n')[1].split(sep=' / ')[0]
    ward_erase = player_data[6].split(sep='\n')[1].split(sep=' / ')[1]
    cs = player_data[7].split(sep='\n')[0]
    
    items = i.find_elements(By.TAG_NAME, 'td')[-1]
    items = items.find_elements(By.TAG_NAME, 'img')
    items = [item.get_attribute('alt') for item in items]
    
    final_data.append([champion, level, summoner_skill, rune, player_name, k, d, a, damage, dpm, dtpm, sight, cs, items])
    
df = pd.DataFrame(final_data, 
                  columns=['champion', 'level', 'summoner_skill', 'rune', 'player_name', 
                           'k', 'd' , 'a', 'damage', 'dpm', 'dtpm', 'sight', 'cs', 'items'])

df

,champion,level,summoner_skill,rune,player_name,k,d,a,damage,dpm,dtpm,sight,cs,items
0,Maokai,17,"[점멸, 순간이동]","[여진, 마법]",MVP ADD,1,6,3,"15,370",386,907,9,289,"[헤르메스의 발걸음, 수정 팔 보호구, 점화석, 정령의 형상, 가고일 돌갑옷, 저녁..."
1,Sejuani,16,"[점멸, 강타]","[여진, 정밀]",MVP Beyond,1,2,2,"5,604",141,513,8,163,"[망가진 초시계, 헤르메스의 발걸음, 기사의 맹세, 예언자의 렌즈]"
2,Syndra,17,"[방어막, 점멸]","[감전, 영감]",MVP Ian,1,3,2,"12,167",305,225,11,360,"[방출의 마법봉, 증폭의 고서, 마법사의 신발, 밴시의 장막, 존야의 모래시계, 망..."
3,Kaisa,17,"[점멸, 순간이동]","[기민한 발놀림, 마법]",MVP Pilot,3,0,1,"22,813",573,341,1,441,"[광전사의 군화, 수은 장식띠, 구인수의 격노검, 루난의 허리케인, 무한의 대검, ..."
4,Braum,14,"[점화, 점멸]","[봉인 풀린 주문서, 결의]",MVP poge,0,1,5,"4,243",107,387,17,88,"[마법무효화의 망토, 헤르메스의 발걸음, 기사의 맹세, 지크의 융합, 예언자의 렌즈]"
5,Chogath,18,"[점멸, 순간이동]","[착취의 손아귀, 정밀]",BtC SoaR,3,0,7,"14,817",372,819,9,364,"[파수꾼의 갑옷, 워모그의 갑옷, 헤르메스의 발걸음, 가고일 돌갑옷, 저녁갑주, 투..."
6,Olaf,18,"[점멸, 강타]","[포식자, 마법]",BtC OnFleek,1,2,5,"10,963",275,793,9,230,"[헤르메스의 발걸음, 칠흑의 양날 도끼, 수호 천사, 예언자의 렌즈]"
7,Fizz,18,"[점화, 점멸]","[감전, 영감]",BtC Dove,5,0,5,"15,679",394,321,12,357,"[라바돈의 죽음모자, 망가진 초시계, 헤르메스의 발걸음, 공허의 지팡이, 마법공학 ..."
8,Xayah,18,"[점멸, 순간이동]","[치명적 속도, 마법]",BtC Hollow,3,2,7,"19,767",496,220,4,450,"[광전사의 군화, 필멸자의 운명, 루난의 허리케인, 헤르메스의 시미터, 무한의 대검..."
9,Morgana,15,"[점화, 점멸]","[콩콩이 소환, 영감]",BtC Joker,0,3,7,"8,214",206,280,28,32,"[망령의 두건, 기동력의 장화, 제어 와드, 점화석, 강철의 솔라리 펜던트, 예언자..."


In [ ]:
#위 과정을 함수로 작성
